In [53]:
!python -V

Python 3.9.12


In [77]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import pickle

In [55]:
def read_greentaxi_dataframe(filename):    
    df = pd.read_parquet(filename)

    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df.loc[((df.duration >= 1) & (df.duration <= 60))]

    df[categorical] = df[categorical].astype(str)
    
    return df


In [56]:
df_train = read_greentaxi_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-01.parquet')
df_val = read_greentaxi_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2024-02.parquet')

In [71]:
len(df_train), len(df_val)

(54373, 51497)

In [72]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [73]:
categorical = ['PU_DO']#'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [59]:
target = 'duration'
Y_train = df_train[target].values
Y_val = df_val[target].values

In [74]:
lr = LinearRegression()
lr.fit(X_train, Y_train)

Y_pred = lr.predict(X_val)

mean_squared_error(Y_val, Y_pred, squared=False)

5.982652114977233

In [78]:
with open('models/lin_reg.bin','wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [75]:
lr = Lasso(alpha=0.001)
lr.fit(X_train, Y_train)

Y_pred = lr.predict(X_val)

mean_squared_error(Y_val, Y_pred, squared=False)

6.583747374286971

In [76]:
lr = Ridge(alpha=0.001)
lr.fit(X_train, Y_train)

Y_pred = lr.predict(X_val)

mean_squared_error(Y_val, Y_pred, squared=False)

8.036870485972175